# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 opencv-python==4.1.2.30 mediapipe

In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                              )

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )

    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                             )

# 3. Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [6]:
DATA_PATH = os.path.join('Dataset') 

signs = np.array(['ا', 'ب', 'ت'])

no_videos = 40

video_length = 30

In [ ]:
for sign in signs:
    end = no_videos
    video = 1
    while video < end + 1:
        try:
            os.makedirs(os.path.join(DATA_PATH, sign, str(video)))
        except:
            end += 1
        video += 1

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
pause = False
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.75, min_tracking_confidence=0.75) as holistic:
    
    for sign in signs:

        print(sign)
        total_no_videos = len(os.listdir(os.path.join(DATA_PATH, sign)))
        for video in range(1, total_no_videos + 1):

            for frame_num in range(1, video_length + 1):
                
                npy_path = os.path.join(DATA_PATH, sign, str(video), str(frame_num))
                if os.path.exists(npy_path + '.npy'):
                    break
                
                if cv2.waitKey(10) & 0xFF == ord('p'):
                    pause = True
                    
                if pause:
                    while True:
                        if cv2.waitKey(10) & 0xFF == ord('p'):
                            pause = False
                            break

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)

                draw_landmarks(image, results)
                
                if frame_num == 1: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, video), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, video), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                np.save(npy_path, keypoints)

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

KeyboardInterrupt: 

In [8]:
label_map = {label:num for num, label in enumerate(signs)}

In [9]:
videos, labels = [], []
for sign in signs:
    total_no_videos = len(os.listdir(os.path.join(DATA_PATH, sign)))
    
    for video in range(1, total_no_videos + 1):
        window = []
        
        for frame_num in range(1, video_length + 1):
            res = np.load(os.path.join(DATA_PATH, sign, str(video), "{}.npy".format(frame_num)))
            res = np.array(res[-126:])
            window.append(res)
        
        videos.append(window)
        labels.append(label_map[sign])

In [10]:
x = np.array(videos)
x.shape

(240, 30, 126)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

(216, 30, 126)
(216, 6)


# 7. Build and Train LSTM Neural Network

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [14]:
log_dir = os.path.join('Log')
tb_callback = TensorBoard(log_dir=log_dir)

In [9]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [17]:
history = model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
7/7 [==============================] - 7s 119ms/step - loss: 1.7856 - categorical_accuracy: 0.2083
Epoch 2/2000
7/7 [==============================] - 1s 110ms/step - loss: 1.7346 - categorical_accuracy: 0.3565
Epoch 3/2000
7/7 [==============================] - 1s 110ms/step - loss: 1.5594 - categorical_accuracy: 0.3704
Epoch 4/2000
7/7 [==============================] - 1s 79ms/step - loss: 1.4577 - categorical_accuracy: 0.4398
Epoch 5/2000
7/7 [==============================] - 1s 107ms/step - loss: 1.6420 - categorical_accuracy: 0.2454
Epoch 6/2000
7/7 [==============================] - 1s 130ms/step - loss: 1.5695 - categorical_accuracy: 0.4630
Epoch 7/2000
7/7 [==============================] - 1s 143ms/step - loss: 1.6428 - categorical_accuracy: 0.4167
Epoch 8/2000
7/7 [==============================] - 1s 132ms/step - loss: 1.6070 - categorical_accuracy: 0.2361
Epoch 9/2000
7/7 [==============================] - 1s 107ms/step - loss: 1.4630 - categorical_accuracy: 

7/7 [==============================] - 1s 105ms/step - loss: 0.2143 - categorical_accuracy: 0.9444
Epoch 146/2000
7/7 [==============================] - 1s 112ms/step - loss: 0.1393 - categorical_accuracy: 0.9583
Epoch 147/2000
7/7 [==============================] - 1s 103ms/step - loss: 0.1620 - categorical_accuracy: 0.9352
Epoch 148/2000
7/7 [==============================] - 1s 151ms/step - loss: 0.9737 - categorical_accuracy: 0.7593
Epoch 149/2000
7/7 [==============================] - 1s 150ms/step - loss: 0.6642 - categorical_accuracy: 0.7870
Epoch 150/2000
7/7 [==============================] - 1s 127ms/step - loss: 0.6461 - categorical_accuracy: 0.7824
Epoch 151/2000
7/7 [==============================] - 1s 166ms/step - loss: 0.3856 - categorical_accuracy: 0.8750
Epoch 152/2000
7/7 [==============================] - 1s 175ms/step - loss: 0.2488 - categorical_accuracy: 0.9074
Epoch 153/2000
7/7 [==============================] - 1s 186ms/step - loss: 0.1636 - categorical_accura

7/7 [==============================] - 1s 84ms/step - loss: 0.0577 - categorical_accuracy: 0.9861
Epoch 290/2000
7/7 [==============================] - 0s 73ms/step - loss: 0.0438 - categorical_accuracy: 0.9861
Epoch 291/2000
7/7 [==============================] - 1s 79ms/step - loss: 0.0403 - categorical_accuracy: 0.9861
Epoch 292/2000
7/7 [==============================] - 1s 72ms/step - loss: 0.0247 - categorical_accuracy: 0.9861
Epoch 293/2000
7/7 [==============================] - 1s 82ms/step - loss: 0.0214 - categorical_accuracy: 0.9907
Epoch 294/2000
7/7 [==============================] - 1s 104ms/step - loss: 0.0161 - categorical_accuracy: 0.9954
Epoch 295/2000
7/7 [==============================] - 1s 83ms/step - loss: 0.0122 - categorical_accuracy: 0.9954
Epoch 296/2000
7/7 [==============================] - 1s 105ms/step - loss: 0.0103 - categorical_accuracy: 1.0000
Epoch 297/2000
7/7 [==============================] - 1s 79ms/step - loss: 0.0081 - categorical_accuracy: 1.0

KeyboardInterrupt: 

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 203,558
Trainable params: 203,558
Non-trai

# 8. Save Weights

In [19]:
model.save('model.h5')

In [20]:
del model

In [11]:
model.load_weights('model.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [12]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [13]:
y_predict = model.predict(x_test)

NameError: name 'x_test' is not defined

In [26]:
y_true = np.argmax(y_test, axis=1).tolist()
y_predict = np.argmax(y_predict, axis=1).tolist()

In [27]:
multilabel_confusion_matrix(y_true, y_predict)

array([[[19,  0],
        [ 0,  5]],

       [[18,  0],
        [ 0,  6]],

       [[22,  1],
        [ 0,  1]],

       [[19,  0],
        [ 1,  4]],

       [[19,  0],
        [ 0,  5]],

       [[22,  0],
        [ 0,  2]]], dtype=int64)

In [28]:
accuracy_score(y_true, y_predict)

0.9583333333333334

# 11. Test in Real Time

In [14]:
def prob_viz(res, signs, input_frame):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), (16,117,245), -1)
        cv2.putText(output_frame, signs[num], (10, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(output_frame, str(round(prob * 100, 2)) + '%', (5, 60+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        keypoints = keypoints[-126:]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            print(signs[np.argmax(res)])
                            sentence.append(signs[np.argmax(res)])
                    else:
                        print(signs[np.argmax(res)])
                        sentence.append(signs[np.argmax(res)])

            if len(sentence) > 10: 
                sentence = sentence[-10:]
            
            image = prob_viz(res, signs, image)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

ل
س
12
1
ل
ب
12
س
12
س
ل
12
س
12
1


In [ ]:
cap.release()
cv2.destroyAllWindows()